In [1]:
import numpy as np
import pandas as pd
import sklearn

In [7]:
print("numpy==%s" %np.__version__)
print("pandas==%s" %pd.__version__)
print("scikit-leran==%s" %sklearn.__version__)

numpy==1.18.1
pandas==0.25.3
scikit-leran==0.23.1


In [2]:
from sklearn.preprocessing import OneHotEncoder
d = pd.read_json("../data/iris.json")

ohe = OneHotEncoder()
x = ohe.fit_transform(d[['Species']])
x = np.array(x.todense()).T
# del d['Species']

for i, species in enumerate(ohe.categories_[0]):
    d[species] = x[i]

In [3]:
sample = np.random.choice(range(150), 130, False)
mask = np.zeros(150, dtype=bool)
mask[sample] = True

d.loc[mask].to_json("../data/train.json")
d.loc[~mask].to_json("../data/test.json")

# Modeling

In [4]:
train = pd.read_json("../data/train.json")
test = pd.read_json("../data/test.json")

In [5]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
X = d.iloc[:,:4].values
y = d.iloc[:,4].values

Xt, Xv, yt, yv = train_test_split(X, y)

In [7]:
pipeline = Pipeline(
    steps=[
        ('normalizer', StandardScaler()), 
        ('power_features', PolynomialFeatures()),
        ('logistic', LogisticRegression(C=1, multi_class='ovr'))
    ])
pipeline.fit(Xt, yt)

Pipeline(steps=[('normalizer', StandardScaler()),
                ('power_features', PolynomialFeatures()),
                ('logistic', LogisticRegression(C=1, multi_class='ovr'))])

In [8]:
pt = pipeline.predict(Xt)
pv = pipeline.predict(Xv)

In [9]:
from sklearn import metrics

print("Confusion train: \n", metrics.confusion_matrix(yt, pt))
print("\nConfusion validation: \n", metrics.confusion_matrix(yv, pv))

Confusion train: 
 [[37  0  0]
 [ 0 38  1]
 [ 0  1 35]]

Confusion validation: 
 [[13  0  0]
 [ 0  8  3]
 [ 0  2 12]]


In [10]:
import pickle

with open("../models/lr1.pickle", "wb") as f:
    pickle.dump(pipeline, f)

In [11]:
import pickle

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split

with open("../models/lr1.pickle", "rb") as f:
    m = pickle.load(f)